In [8]:
import sys
sys.path.append('app')

In [9]:
import os
import sqlite3
from database.crud import __tablefilter

# PATH = os.environ.get("DATA_PATH")
# PATH = "var/lib/data"

PATH = './data'

DATABASE_PATH = os.path.join(os.path.abspath(PATH), 'financedata.db')

connection = sqlite3.connect(DATABASE_PATH, check_same_thread=True)

cursor = connection.cursor()

In [10]:
from datetime import datetime 

def submissions_form_get(connection, filter:dict, columns:list=None, date_from:str=None, date_to:str=None, item_limit:int=1000) -> list[dict]:
    """
    Search for company filing by filter and return the result as a list of dictionary.
    
    :param:
        connection: sqlite connection object
        filter: The filter (in dict) to search for
        columns: List of columns to return (default is all columns)
        date_from: filter date from 
        date_to: filterr date to
    :return: Dictionary containing company info (return list if have many result) if found, otherwise None
    """
    cursor = connection.cursor()
    
    filter = {} if filter is None else list(filter.items())
    
    condition = []
    if filter is not None or len(filter) != 0:
        for q, v in filter:
            if v is not None:
                if hasattr(v, '__iter__') and not isinstance(v, str):
                    condition.append(f"{q} IN {str(tuple(v))}")
                else:
                    condition.append("{} = \"{}\"".format(q, v))
                
        condition = f" AND ".join(condition)    
    
        
    date_from = datetime.strftime(datetime.strptime(date_from, "%Y-%m-%d"), "%Y-%m-%d") if date_from is not None else None
    date_to = datetime.strftime(datetime.strptime(date_to, "%Y-%m-%d"), "%Y-%m-%d") if date_to is not None else None
    
    if date_from is not None:
        datefrom_cont = f"date(filing_date) > date(\"{date_from}\")"
        condition = condition + f" AND {datefrom_cont}" if len(condition) > 0 else datefrom_cont
    if date_to is not None:
        dateto_cont = f"date(filing_date) < date(\"{date_to}\")"
        condition = condition + f" AND {dateto_cont}" if len(condition) > 0 else dateto_cont
        
    if len(condition) == 0:
        condition = "TRUE"

    column = """
        ticker, companyInfo.cik, primary_doc_description, primary_docment,
        accession_number, filing_date, report_date, acceptance_date_time, 
        index_url, primary_docment_url, act, form
        """ if columns is None else ','.join(columns)
    
    query = f"""
        SELECT {column}
        FROM submissionForm INNER JOIN companyInfo ON submissionForm.cik = companyInfo.cik
        WHERE {condition}
        """
    
    print(query)
    
    q_res = cursor.execute(query)
    data = q_res.fetchmany(item_limit)
    
    columns = [ col[0] for col in q_res.description]
    
    if len(data) == 0:
        return None
    
    datadict = [ dict(zip(columns, r)) for r in data ]
    return datadict

# submissions_form_get(
#     connection, 
#     filter={
#         "ticker": None,
#         "cik": None,
#         "accession_number": None,
#         "act": None,
#         "form": ["10-Q", "10-K"] ,
#     }, 
#     columns=None, 
#     date_from=None, 
#     date_to=None, 
#     item_limit=1000
#     )

In [17]:
query = f"""
    SELECT ticker, name, entity_type
    FROM companyInfo
    WHERE entity_type != "operating" AND entity_type != "other"
    """
    
q_res = cursor.execute(query)
q_res.fetchmany(500)

[('HFRO', 'HIGHLAND OPPORTUNITIES & INCOME FUND', 'investment'),
 ('EAD', 'ALLSPRING INCOME OPPORTUNITIES FUND', 'investment'),
 ('KTF', 'DWS MUNICIPAL INCOME TRUST', 'investment'),
 ('ERC', 'ALLSPRING MULTI-SECTOR INCOME FUND', 'investment'),
 ('HGLB', 'HIGHLAND GLOBAL ALLOCATION FUND', 'investment'),
 ('FMN', 'Federated Hermes Premier Municipal Income Fund', 'investment'),
 ('VFL', 'abrdn National Municipal Income Fund', 'investment'),
 ('KSM', 'DWS STRATEGIC MUNICIPAL INCOME TRUST', 'investment'),
 ('ERH', 'ALLSPRING UTILITIES & HIGH INCOME FUND', 'investment'),
 ('SELF', 'Global Self Storage, Inc.', 'investment'),
 ('FXBY', 'FOXBY CORP.', 'investment'),
 ('HFRO-PA', 'HIGHLAND OPPORTUNITIES & INCOME FUND', 'investment')]